# Mashup Semantic Web project:
Databases:
- GeoNames(http://www.geonames.org/)
- DBPedia()
Problem:
- We give some name of two cities, then we want to calculate a distance beetwen them and then get some info about country and specific region that they are in

## <span style="color:cyan">Importy i funkcje pomocnicze

In [94]:
import SPARQLWrapper
import pgeocode
import pandas as pd
import math
from collections import Counter
import requests
import tkinter as tk
from tkinter import messagebox, ttk

In [95]:
def get_geoname_id_by_name(city_name, country, username):
    url = "http://api.geonames.org/searchJSON"
    params = {
        'q': city_name,
        'country': country,
        'maxRows': 1,
        'username': username
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if 'geonames' in data and len(data['geonames']) > 0:
                geoname_id = data['geonames'][0].get('geonameId', None)
                if geoname_id:
                    return geoname_id
                return "GeoNames ID not found in response."
            else:
                return f"No data found for city {city_name}."
        else:
            return f"Error querying GeoNames API: HTTP {response.status_code} - {response.text}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [96]:
def get_city(city1):
    country_code1, name1 = city1
    cnt1_nom = pgeocode.Nominatim(country_code1)
    data1 = cnt1_nom.query_location(name1, top_k=20,fuzzy_threshold=40)
    best_match1 = Counter(data1.place_name).most_common(1)[0][0]
    data1 = data1[data1['place_name'] == best_match1]
    data1 = data1.drop_duplicates(subset='place_name', keep='first')
    
    data1['geoname_id'] = get_geoname_id_by_name(data1['place_name'], data1['country_code'], 'wrex')
    return data1

In [97]:
def calculate_dist(city1, city2):
    name1 = city1['place_name'].values[0]
    name2 = city2['place_name'].values[0]
    
    lat1, lon1 = tuple(*city1[['latitude', 'longitude']].values)
    lat2, lon2 = tuple(*city2[['latitude', 'longitude']].values)
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371.0
    distance = R * c
    return name1,name2,distance

In [98]:
def get_city_museums(city_id):
    if type(city_id) == pd.DataFrame:
        zipcode = city_id['geoname_id'].values[0]
    elif type(city_id) == str:
        zipcode = city_id
    else:
        raise TypeError("Zipcode must be either a string or a DataFrame")
    wikidata_spraql = SPARQLWrapper.SPARQLWrapper("https://query.wikidata.org/sparql")
    wikidata_spraql.setReturnFormat(SPARQLWrapper.JSON)
    #city_URI = 'http://www.wikidata.org/entity/Q31487'
    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    
    SELECT ?city_uri WHERE {{
      ?city_uri wdt:P1566 '{zipcode}'.
      }}
    """
    wikidata_spraql.setQuery(query)
    city_URI = wikidata_spraql.query().convert()['results']['bindings'][0]['city_uri']['value'] 
    #wd:Q33506
    query2 = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    
    SELECT ?museumLabel ?street_address WHERE {{
      ?museum wdt:P131 <{city_URI}>;
      wdt:P6375 ?street_address;
      rdfs:label ?museumLabel;
      wdt:P31 ?class.
      ?class wdt:P279 ?subclass.
      FILTER(LANG(?museumLabel)="pl" && (?subclass = wd:Q33506 || ?class = wd:Q33506)).
    }}
    """
    #*
    # 1)Muzeum jest w miescie o danym URI
    # 2) bierzemy sobie przy jakiej ulicy jest to muzeum
    # 3) bierzemy sobiejego opis po polsku
    # 4 i 5) sprawdzamy czy instancja ma podklase muzeum 
    # *#
    wikidata_spraql.setQuery(query2)
    museums = wikidata_spraql.query().convert()['results']['bindings']
    df = pd.DataFrame({
        'museum_name': [museum['museumLabel']['value'] for museum in museums],
        'address': [museum['street_address']['value'] for museum in museums]
    })
    df.drop_duplicates(subset='museum_name', keep='first', inplace=True, ignore_index=True)
    return df

## <span style="color:cyan">Zmienne pomocnicze

In [99]:
prefix = 'https://www.wikidata.org/wiki/'
POLAND_CODE = 'PL'
city_krakow = (POLAND_CODE, "Kraków")
city_katowice = (POLAND_CODE, "Katowice")
city_warszawa = (POLAND_CODE, "Warszawa")
city_szczecin = (POLAND_CODE, "Szczecin")

## <span style="color:cyan">Kod z funkcjonalnościami

In [100]:
cracov_city_data = get_city(city_krakow)
cracov_city_data

,country_code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy,geoname_id
24588,PL,30-001,Kraków,Lesser Poland,77,Kraków,1261.0,Kraków,126101.0,50.0759,19.941,6,3094802


In [101]:
katowice_city_data = get_city(city_katowice)
katowice_city_data

,country_code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy,geoname_id
52665,PL,40-001,Katowice,Silesia,83,Katowice,2469.0,Katowice,246901.0,50.1965,18.9717,6,3096472


In [102]:
warszawa_city_data = get_city(city_warszawa)
warszawa_city_data

,country_code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy,geoname_id
34494,PL,00-002,Warszawa,Mazovia,78,Warszawa,1465.0,Warsaw,146501.0,52.2358,21.0101,6,756135


In [103]:
szczecin_city_data = get_city(city_szczecin)
szczecin_city_data

,country_code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy,geoname_id
7774,PL,87-860,Szczecin,Kujawsko-Pomorskie,73,Powiat włocławski,418.0,Gmina Chodecz,41806.0,52.3664,19.0099,6,3083829


In [104]:
get_city_museums(katowice_city_data)

,museum_name,address
0,Harcerskie Muzeum Etnograficzne,"ul. Nasypowa 16, 40-551 Katowice"
1,Izba Śląska,"Plac Pod Lipami 3-3a, 40-476 Katowice"
2,Muzeum Barbary i Stanisława Ptaków,"ul. M. Kopernika 11, 40-064 Katowice"
3,Muzeum Historii Katowic,"Ks. J. Szafranka 9, 40-025 Katowice"
4,Muzeum - Izba Pamięci Kopalni Wujek w Katowicach,"ul. Wincentego Pola 38, 40-596 Katowice"
5,Muzeum Historii Komputerów i Informatyki,"pl. Sejmu Śląskiego 2, 40-032 Katowice"
6,Muzeum Historii Gitary w Katowicach,"ul. Krzywa 4, 40-061 Katowice"
7,Muzeum Śląskie w Katowicach,"Dobrowolskiego 1, 40-205 Katowice"
8,Katowicki Skład Neonów,"Plac Sejmu Śląskiego 2, 40-032 Katowice"


In [105]:
get_city_museums(warszawa_city_data)

,museum_name,address
0,Muzeum Łowiectwa i Jeździectwa w Warszawie,"ul. Agrykola 1, 00-460 Warszawa"
1,Muzeum Warszawy,"rynek Starego Miasta 28-42, 00-272 Warszawa"
2,Muzeum Plakatu w Wilanowie,"ul. St. Kostki Potockiego 10/16, 02-958 Warszawa"
3,X Pawilon Cytadeli Warszawskiej,Skazańców 25
4,Muzeum Farmacji im. mgr Antoniny Leśniewskiej,Piwna 33
5,Muzeum Historii Polski w Warszawie,"ul. Dymińska 13, 01-519 Warszawa"
6,Fabryka Czekolady E. Wedel w Warszawie,"03-822 Warszawa, aleja Emila Wedla 5"
7,Muzeum Kowalstwa w Warszawie,"ul. Przy Grobli 84, 02-749 Warszawa"
8,Muzeum Ordynariatu Polowego,ul. Długa 13/15 00-238 Warszawa
9,Muzeum Wodociągów i Kanalizacji,"ul. Koszykowa 81, 02-012 Warszawa"


In [106]:
get_city_museums(cracov_city_data)

,museum_name,address
0,Muzeum Narodowe w Krakowie,"Aleja 3 Maja 1, 30-062 Kraków"
1,Synagoga Stara w Krakowie,"31-053 Kraków, ul. Szeroka 24"
2,Muzeum Witrażu w Krakowie,"al. Krasińskiego 23, 31-111 Kraków"
3,Ulica Pomorska,"Pomorska 2, 30-039 Kraków"
4,"Ekspozycja ""Dawne Narzędzia Tortur"" w Krakowie",Rynek Główny 29
5,Galeria Krzysztofory,ul. Szczepańska 2
6,Aquarium i Muzeum Przyrodnicze w Krakowie,"31-049 Kraków, Świętego Sebastiana 9"
7,Centrum Edukacji Przyrodniczej,"ul. Gronostajowa 5, 30-387 Kraków"
8,Muzeum Przyrodnicze Instytutu Systematyki i Ew...,"Świętego Sebastiana 9, 31-049 Kraków"
9,Apteka Pod Orłem w Krakowie,"plac Bohaterów Getta 18, 30-547 Kraków"


In [107]:
get_city_museums(szczecin_city_data)

,museum_name,address
0,Gmach Główny Muzeum Narodowego w Szczecinie,Wały Chrobrego 3


## <span style="color:cyan">Wizualizacja Projektu

In [108]:
# Wizualizacja Projektu: Funkcje Pomocnicze
def validate_input(city_name):
    if not city_name or len(city_name) > 50 or len(city_name.split()) > 50:
        return False
    return True

def is_valid_dataframe(df):
    return isinstance(df, pd.DataFrame) and not df.empty

def search_city():
    city_name = city_input.get().strip()
    if not validate_input(city_name):
        messagebox.showerror("Błąd", "Wprowadź poprawną nazwę miasta (1-50 znaków).")
        return

    try:
        city_data = get_city((POLAND_CODE, city_name))
        if not is_valid_dataframe(city_data):
            raise ValueError(f"Brak danych dla miasta: {city_name}")

        museums_data = get_city_museums(city_data)
        if not is_valid_dataframe(museums_data):
            raise ValueError(f"Brak danych o muzeach dla miasta: {city_name}")

        display_city_results(city_name, city_data, museums_data)
    except Exception as e:
        messagebox.showerror("Błąd", str(e))

def display_city_results(city_name, city_data, museums_data):
    clear_window()

    city_label = tk.Label(root, text=f"Informacje o mieście: {city_name}", font=("Helvetica", 16), bg="#E6F2FF", fg="black")
    city_label.pack(pady=10)

    if is_valid_dataframe(city_data):
        city_data = city_data.astype(str)

        city_table = ttk.Treeview(root, columns=city_data.columns.tolist(), show="headings", height=3)
        for col in city_data.columns:
            city_table.heading(col, text=col)
            city_table.column(col, anchor="center", width=125)

        for _, row in city_data.iterrows():
            city_table.insert("", "end", values=row.tolist())

        city_table.pack(pady=10)
    else:
        no_data_label = tk.Label(root, text="Brak dostępnych danych dla miasta.", font=("Helvetica", 12), fg="red", bg="#E6F2FF")
        no_data_label.pack(pady=10)

    additional_info_button = tk.Button(root, text="Dodatkowe Informacje o muzeach", command=lambda: display_museum_results(city_name, museums_data), bg="#4DB8FF", fg="white")
    additional_info_button.pack(pady=5)

    search_again_button = tk.Button(root, text="Wyszukaj ponownie", command=initialize_ui, bg="#4DB8FF", fg="white")
    search_again_button.pack(pady=5)

def display_museum_results(city_name, museums_data):
    clear_window()

    museum_label = tk.Label(root, text=f"Muzea w mieście: {city_name}", font=("Helvetica", 16), bg="#E6F2FF", fg="black")
    museum_label.pack(pady=10)

    if is_valid_dataframe(museums_data):
        museums_data = museums_data.astype(str)

        museum_table = ttk.Treeview(root, columns=museums_data.columns.tolist(), show="headings", height=10)
        for col in museums_data.columns:
            museum_table.heading(col, text=col)
            museum_table.column(col, anchor="center", width=500)

        for _, row in museums_data.iterrows():
            museum_table.insert("", "end", values=row.tolist())

        museum_table.pack(pady=10)
    else:
        no_data_label = tk.Label(root, text="Brak dostępnych danych o muzeach.", font=("Helvetica", 12), fg="red", bg="#E6F2FF")
        no_data_label.pack(pady=10)

    search_again_button = tk.Button(root, text="Wyszukaj ponownie", command=initialize_ui, bg="#4DB8FF", fg="white")
    search_again_button.pack(pady=5)

def clear_window():
    for widget in root.winfo_children():
        widget.destroy()

def initialize_ui():
    clear_window()

    main_frame = tk.Frame(root, bg="#E6F2FF")
    main_frame.pack(pady=20)

    header = tk.Label(main_frame, text="Mashup Semantic Web Project\nGrupa 🐢", font=("Helvetica", 24, "bold"), bg="#E6F2FF", fg="black")
    header.pack(pady=10)

    form_frame = tk.Frame(main_frame, bg="#E6F2FF")
    form_frame.pack(pady=10)

    city_label = tk.Label(form_frame, text="Podaj miasto polskie:", font=("Helvetica", 14), bg="#E6F2FF", fg="black")
    city_label.grid(row=0, column=0, padx=5, pady=5)

    global city_input
    city_input = tk.Entry(form_frame, font=("Helvetica", 14), width=25)
    city_input.grid(row=0, column=1, padx=5, pady=5)

    search_button = tk.Button(form_frame, text="Szukaj", command=search_city, bg="#4DB8FF", fg="white")
    search_button.grid(row=0, column=2, padx=5, pady=5)

    suggested_frame = tk.Frame(main_frame, bg="#E6F2FF")
    suggested_frame.pack(pady=10)

    suggested_label = tk.Label(suggested_frame, text="Proponowane miasta do przejrzenia:", font=("Helvetica", 14), bg="#E6F2FF", fg="black")
    suggested_label.pack(pady=5)

    suggested_cities = ["Kraków", "Warszawa", "Katowice", "Toruń", "Szczecin"]
    for city in suggested_cities:
        city_button = tk.Button(suggested_frame, text=city, bg="#4DB8FF", fg="white",
                                command=lambda c=city: fill_city_input(c))
        city_button.pack(pady=2)

def fill_city_input(city_name):
    city_input.delete(0, tk.END)
    city_input.insert(0, city_name)

# Wizualizacja Projektu: Wizualizacja Okienkowa
root = tk.Tk()
root.title("Mashup Semantic Web Project, Grupa 🐢")
root.geometry("1750x500")
root.configure(bg="#E6F2FF")

initialize_ui()
root.mainloop()